In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import FeatureHasher
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import *
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Load data

In [15]:
df_train = spark.read.csv("data/application_train.csv",inferSchema="true", header="true")
#df_test = spark.read.csv("data/application_test.csv",inferSchema="true", header="true")
#df_bbal = spark.read.csv("data/bureau_balance.csv",inferSchema="true", header="true")
#df_cbal = spark.read.csv("data/credit_card_balance.csv",inferSchema="true", header="true")
#df_inspay = spark.read.csv("data/installments_payments.csv",inferSchema="true", header="true")
#df_pos = spark.read.csv("data/POS_CASH_balance.csv",inferSchema="true", header="true")
#df_prevap = spark.read.csv("data/previous_application.csv",inferSchema="true", header="true")

# Make RDD

In [16]:
df_train_rdd = df_train.rdd
#df_test = df_test.rdd
#df_bbal = df_bbal.rdd
#df_cbal = df_cbal.rdd
#df_inspay = df_inspay.rdd
#df_pos = df_pos.rdd
#df_prevap = df_prevap.rdd

In [17]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="TARGET", outputCol="indexedLabel").fit(df_train)

#featureIndexer = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df_train) for column in list(set(df_train.columns))]
hasher = FeatureHasher(inputCols=[column for column in list(set(df_train.columns))],outputCol="indexedFeatures")

In [18]:
# Split the data into training and test sets (30% held out for testing)
trainingData, testData = df_train.randomSplit([0.7, 0.3])

# Train a RandomForest model.
featurized = hasher.transform(df_train)
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
#pipeline = Pipeline().setStages([labelIndexer] + featureIndexer + [rf, labelConverter])
pipeline = Pipeline(stages=[labelIndexer, hasher, rf, labelConverter])

In [19]:
# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

In [21]:
# Make predictions.
predictions = model.transform(testData)

In [26]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

print(rf)  # summary only

Test Error = 0.0808012
RandomForestClassifier_0d4b2593447a
